<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

импортируем необходимые библиотеки и сохраним данные клиентов страховой компании «Хоть потоп»:

In [3]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [4]:
data=pd.read_csv('/datasets/insurance.csv')

выведем первые 10 строк таблицы:

In [5]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


и выведем информацию о данных:

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


в данных не обнаружено пропусков

проверим данные на явные дубликаты:

In [7]:
data.duplicated().sum()

153

переведем столбцы Возраст и Зарплата в формат int:

In [8]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

и проверим что столбцы поменяли тип данных:

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


и переименуем столбцы согласно правилам синтаксиса, чтобы названия были на английском языке:

In [10]:
data = data.rename(columns={'Пол':'sex','Возраст':'age','Зарплата':'income','Члены семьи':'family_members','Страховые выплаты':'insurance_payments'}) 

In [11]:
data.columns

Index(['sex', 'age', 'income', 'family_members', 'insurance_payments'], dtype='object')

проверим что в данных нет аномалий, например 3го пола или нулевого возраста:

In [12]:
data['sex'].value_counts()

0    2505
1    2495
Name: sex, dtype: int64

In [13]:
data['age'].value_counts()

19    223
25    214
31    212
26    211
22    209
27    209
32    206
28    204
29    203
30    202
23    202
21    200
20    195
36    193
33    191
24    182
35    179
34    177
37    147
39    141
38    139
41    129
18    117
40    114
42     93
43     77
44     74
45     73
46     60
47     47
49     37
50     27
48     26
52     22
51     21
53     11
55      9
54      7
56      5
59      3
60      2
57      2
58      2
65      1
61      1
62      1
Name: age, dtype: int64

In [14]:
data['income'].value_counts()

45800    29
37100    28
41500    27
43200    27
38800    26
         ..
61200     1
15200     1
71600     1
14300     1
63200     1
Name: income, Length: 524, dtype: int64

In [15]:
data['family_members'].value_counts()

1    1814
0    1513
2    1071
3     439
4     124
5      32
6       7
Name: family_members, dtype: int64

In [16]:
data['insurance_payments'].value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: insurance_payments, dtype: int64

аномалий не обнаружено, перейдем к заданию

## Умножение матриц

Проверим изменится ли качество линейной регрессии, если умножить признаки на обратимую матрицу. 

подставим в формулу 
$
w = (X^T X)^{-1} X^T y
$
значение
$
X_{new}=X*P
$
:
$$
w_{new}=((X*P)^T X*P)^{-1} (X*P)^T y = (P^T*X^T*X*P)^{-1} (X*P)^T y = (P)^{-1}(P^T*X^T*X)^{-1}(P)^T(X)^T y = P^{-1}(X^T*X)^{-1}P^{-T}(P)^T(X)^T y = P^{-1}(X^T*X)^{-1}(X)^T y = 
$$

так как $w = (X^T X)^{-1} X^T y$ :
$$
w_{new}=P^{-1}*w
$$
А значит:

$$
a_{new}=X_{new}w_{new}=X*P*P^{-1}w = Xw=a
$$

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** нет, не изменится

**Обоснование:** ...

## Алгоритм преобразования

**Алгоритм**

в пункт 2 мы доказали что при умножении матрицы признаков на матрицу P. Методом numpy.random.normal() можем сознать случайную матрицу, умножить матрицу признаков на матрицу P. качество моделей машинного обучения не ухудшится, а персональные данные клиентов будут защищены.

**Обоснование**

...

## Проверка алгоритма

выделим признаки:

In [17]:
features = data.drop(['insurance_payments'], axis=1)

выделим целевой признак:

In [18]:
target=data['insurance_payments']

преобразуем признаки в матрицу:

In [19]:
features_matrix=np.array(features)

In [20]:
features_matrix

array([[    1,    41, 49600,     1],
       [    0,    46, 38000,     1],
       [    0,    29, 21000,     0],
       ...,
       [    0,    20, 33900,     2],
       [    1,    22, 32700,     3],
       [    1,    28, 40600,     1]])

создадим случайную матрицу размером 4 на 4:

In [21]:
random_matrix = np.random.normal(size=(4,4))
random_matrix

array([[ 0.57858074,  0.26354115, -0.09263432,  1.60096303],
       [ 0.98126828,  0.47618628, -2.24233542, -0.92187167],
       [-0.04583322, -0.10971565, -0.25526947,  0.70416735],
       [ 0.37980628, -1.22785818, -0.48420659, -1.93133992]])

найдем обратную матрицу к random_matrix:

In [22]:
ainv = inv(random_matrix)

In [23]:
ainv

array([[ 1.25129638,  0.03987945, -1.61728377,  0.4285507 ],
       [-0.12603153,  0.26932717, -1.11023741, -0.63782135],
       [ 0.43114799, -0.33909839, -1.22968798,  0.07090987],
       [ 0.21810473, -0.0783681 ,  0.69608802, -0.0457789 ]])

ошибки не произошло, значит матрица обратима.

теперь умножим матрицу признаков `features_matrix` на матрицу `random_matrix`

In [24]:
features_matrix_new = features_matrix @ random_matrix
features_matrix_new

array([[ -2232.13734266,  -5423.33707097, -12753.87833957,
         34888.57337013],
       [ -1696.14422592,  -4148.51810536,  -9803.87152228,
         26714.02180615],
       [  -934.04084709,  -2290.21931203,  -5425.68661175,
         14760.78004   ],
       ...,
       [ -1533.36119161,  -3712.29262923,  -8699.45017812,
         23848.97300086],
       [ -1475.4404036 ,  -3580.64579004,  -8398.18832503,
         23001.79806242],
       [ -1832.39484722,  -4442.08661511, -10427.30274286,
         28563.05156538]])

перейдем к обучению модели. выделим тренировочную выборку и тестирующую

In [25]:
data_train,data_test = train_test_split(data, test_size=0.25, random_state=42)

In [26]:
data_train.shape

(3750, 5)

In [27]:
data_test.shape

(1250, 5)

выделим признаки и целевой признак у тренировочной выборки и тестирующей выборки:

In [28]:
features_train = data_train.drop(['insurance_payments'], axis=1)

In [29]:
target_train = data_train['insurance_payments']

In [30]:
features_test = data_test.drop(['insurance_payments'], axis=1)

In [31]:
target_test = data_test['insurance_payments']

обучим модель линейной регрессии на тренировочной выборке:

In [32]:
model=LinearRegression() 

In [33]:
model.fit(features_train,target_train)

LinearRegression()

получим предсказания модели на тестовой выборке

In [34]:
predictions_test = model.predict(features_test)

In [35]:
r2_score(target_test, predictions_test)

0.4254778535754763

Теперь обучим модель на преобразованный данных используя матрицу features_matrix_new

In [36]:
data_train_new,data_test_new = train_test_split(features_matrix_new, test_size=0.25, random_state=42)

In [37]:
data_train_new.shape

(3750, 4)

In [38]:
data_test_new.shape

(1250, 4)

In [39]:
features_train_new, features_test_new, target_train_new, target_test_new = train_test_split(
    features_matrix_new, target, train_size=0.75, test_size=0.25, random_state=42)

In [40]:
model_new=LinearRegression() 

In [41]:
model_new.fit(features_train_new,target_train_new)

LinearRegression()

In [42]:
predictions_test_new = model_new.predict(features_test_new)

In [43]:
r2_score(target_test_new, predictions_test_new)

0.42547785357548773

Показатель r2 несущественно отличается до и после преобразования